In [82]:
import pandas as pd
training = pd.read_excel('training.xlsx')
scoring = pd.read_excel("scoring.xlsx")

In [83]:
training

,Date,Vehicle Category,GVWR Class,Fuel Type,Model Year,Fuel Technology,Electric Mile Range,Number of Vehicles Registered at the Same Address,Region,Vehicle Population
0,2019,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,≥4,Statewide,395883
1,2020,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,1,Statewide,370954
2,2021,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,1,Statewide,349406
3,2019,P,Not Applicable,Gasoline,2019.0,ICE,Not Applicable,≥4,Statewide,348475
4,2019,P,Not Applicable,Gasoline,2018.0,ICE,Not Applicable,≥4,Statewide,333296
...,...,...,...,...,...,...,...,...,...,...
41048,2019,B,Not Applicable,Diesel,1983.0,ICE,Not Applicable,1,Statewide,1
41049,2019,B,Not Applicable,Diesel,1980.0,ICE,Not Applicable,2,Statewide,1
41050,2019,B,Not Applicable,Diesel,1978.0,ICE,Not Applicable,3,Statewide,1
41051,2019,B,Not Applicable,Diesel,1976.0,ICE,Not Applicable,2,Statewide,1


In [84]:
scoring

,Date,Vehicle Category,GVWR Class,Fuel Type,Model Year,Fuel Technology,Electric Mile Range,Number of Vehicles Registered at the Same Address,Region,Vehicle Population
0,2024,P,Not Applicable,Gasoline,2020.0,ICE,Not Applicable,≥4,Statewide,316065
1,2024,P,Not Applicable,Gasoline,2021.0,ICE,Not Applicable,≥4,Statewide,315986
2,2024,P,Not Applicable,Gasoline,2022.0,ICE,Not Applicable,≥4,Statewide,306487
3,2024,P,Not Applicable,Gasoline,2024.0,ICE,Not Applicable,≥4,Statewide,284754
4,2024,P,Not Applicable,Gasoline,2023.0,ICE,Not Applicable,≥4,Statewide,284153
...,...,...,...,...,...,...,...,...,...,...
7541,2024,B,Not Applicable,Diesel,1989.0,ICE,Not Applicable,3,Statewide,1
7542,2024,B,Not Applicable,Diesel,1988.0,ICE,Not Applicable,2,Statewide,1
7543,2024,B,Not Applicable,Diesel,1988.0,ICE,Not Applicable,3,Statewide,1
7544,2024,B,Not Applicable,Diesel,1987.0,ICE,Not Applicable,2,Statewide,1


In [ ]:
# Update missing values in Model Year to use mode from Fuel Type, Fuel Technology subcategories
mode_per_group = training.groupby(["Fuel Type", "Fuel Technology"])['Model Year'].transform(lambda x: pd.Series.mode(x).iloc[0] if not x.mode().empty else x)
training['Model Year'] = training['Model Year'].fillna(mode_per_group)

training = training.rename(columns={
    'Date': 'year',
    'Vehicle Category': 'vehicleCategory',
    'GVWR Class': 'gvwrClass',
    'Fuel Type': 'fuelType',
    "Model Year": "modelYear",
    "Fuel Technology": "fuelTechnology",
    "Electric Mile Range": "electricMileRange",
    "Number of Vehicles Registered at the Same Address": "numVehiclesReg",
    "Vehicle Population:": "vehiclePopulation"
})

training = training.drop(columns=['Region'])

'''
training["gvwrClassKN"] = training["gvwrClass"].apply(lambda x: 1 if x == "Unknown" else 0)
training["gvwrClass"] = training["gvwrClass"].replace("Unknown", -1)
training["gvwrClass"] = training["gvwrClass"].replace("Not Applicable", 0)'
'''

training["numVehiclesReg"] = training["numVehiclesReg"].apply(lambda x: 4 if x == "≥4" else x)
training["numVehiclesReg"] = training["numVehiclesReg"].replace("Unknown", -1)

training = pd.get_dummies(training,
                            columns=["gvwrClass", "vehicleCategory", "fuelType", "fuelTechnology", "electricMileRange"],
                            prefix=["gvwrClass", "vehicleCategory", "fuelType", "fuelTech", "electricRange"],
                            prefix_sep="_")

y_scoring_actual = training['Vehicle Population'] 

training = training.drop(columns=['Vehicle Population'])
if 'fuelType_Unknown' not in training.columns:
    training['fuelType_Unknown'] = 0

C:\Users\ginap\AppData\Local\Temp\ipykernel_17548\1113896877.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  training["numVehiclesReg"] = training["numVehiclesReg"].replace("Unknown", -1)


In [88]:
training

,year,modelYear,numVehiclesReg,gvwrClass_1,gvwrClass_2,gvwrClass_3,gvwrClass_4,gvwrClass_5,gvwrClass_6,gvwrClass_7,...,fuelTech_BEV,fuelTech_FCEV,fuelTech_ICE,fuelTech_PHEV,electricRange_0 to 50 miles,electricRange_101 to 150 miles,electricRange_51 to 100 miles,electricRange_>150 miles,electricRange_Not Applicable,electricRange_Unknown
0,2019,2020.0,4,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
1,2020,2020.0,1,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
2,2021,2020.0,1,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
3,2019,2019.0,4,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
4,2019,2018.0,4,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41048,2019,1983.0,1,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
41049,2019,1980.0,2,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
41050,2019,1978.0,3,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
41051,2019,1976.0,2,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,False
